# 模型，提示和输出解释器


**目录**
* 获取你的OpenAI API Key
* 直接调用OpenAI的API
* 通过LangChain进行的API调用：
  * 提示（Prompts）
  * [模型（Models)](#model)
  * 输出解析器（Output parsers）
  

## 获取你的OpenAI API Key

登陆[OpenAI账户获取你的API Key](https://platform.openai.com/account/api-keys) 

In [26]:
# 下载需要的包python-dotenv和openai
# 如果你需要查看安装过程日志，可删除 -q 
# !pip install -q python-dotenv
# !pip install -q openai

In [27]:
# import sys
# sys.path.append("D:\\software\\conda\\envs\\torch2\\Lib\\site-packages")

import os
# import openai
from openai import OpenAI


api_key = os.environ.get('OPENAI_API_KEY')
base_url = os.environ.get('OPENAI_BASE_URL')
# print("OpenAI API Key:", api_key)


client = OpenAI(
    api_key = api_key,
    base_url= base_url
)

# 运行此API配置，需要将目录中的.env中api_key替换为自己的
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
# 导入 OpenAI API_KEY


## Chat API：OpenAI

我们先从直接调用OpenAI的API开始。

`get_completion`函数是基于`openai`的封装函数，对于给定提示（prompt）输出相应的回答。其包含两个参数
   
   - `prompt` 必需输入参数。 你给模型的提示，可以是一个问题，可以是你需要模型帮助你做的事（改变文本写作风格，翻译，回复消息等等）。
   - `model` 非必需输入参数。默认使用gpt-3.5-turbo。你也可以选择其他模型。
   
这里的提示对应我们给chatgpt的问题，函数给出的输出则对应chatpgt给我们的答案。

In [28]:
def get_completion(prompt, model="gpt-3.5-turbo-0125"):
    
    messages = [{"role": "user", "content": prompt}]
    
    response =  client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message.content

### 一个简单的例子

我们来一个简单的例子 - 分别用中英文问问模型

- 中文提示(Prompt in Chinese)： `1+1是什么？`
- 英文提示(Prompt in English)： `What is 1+1?`

In [29]:
get_completion("1+1是什么？")

'1+1等于2。'

In [30]:
get_completion("What is 1+1?")

'1+1 equals 2.'

### 复杂一点例子

上面的简单例子，模型`gpt-3.5-turbo`对我们的关于1+1是什么的提问给出了回答。

现在我们来看一个复杂一点的例子： 

假设我们是电商公司员工，我们的顾客是一名海盗A，他在我们的网站上买了一个榨汁机用来做奶昔，在制作奶昔的过程中，奶昔的盖子飞了出去，弄得厨房墙上到处都是。于是海盗A给我们的客服中心写来以下邮件：`customer_email`

In [31]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

我们的客服人员对于海盗的措辞表达觉得有点难以理解。 现在我们想要实现两个小目标：

- 让模型用美式英语的表达方式将海盗的邮件进行翻译，客服人员可以更好理解。*这里海盗的英文表达可以理解为英文的方言，其与美式英语的关系，就如四川话与普通话的关系。
- 让模型在翻译是用平和尊重的语气进行表达，客服人员的心情也会更好。

根据这两个小目标，定义一下文本表达风格：`style`

In [32]:
# 美式英语 + 平静、尊敬的语调
style = """American English \
in a calm and respectful tone , at last translate to Simplified Chinese
"""

下一步需要做的是将`customer_email`和`style`结合起来构造我们的提示:`prompt`

In [33]:
# 要求模型根据给出的语调进行转化
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone , at last translate to Simplified Chinese
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



`prompt` 构造好了，我们可以调用`get_completion`得到我们想要的结果 - 用平和尊重的语气，美式英语表达的海岛邮件

In [34]:
response = get_completion(prompt)

In [35]:
response

"Oh man, I'm really upset that my blender lid flew off and made a mess of my kitchen walls with smoothie! And on top of that, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy!\n\n哦，天啊，我真的很生气，我的搅拌机盖飞了出去，把我的厨房墙壁弄得到处都是果昔！更糟糕的是，保修不包括清理厨房的费用。伙计，我现在真的需要你的帮助！"

对比语言风格转换前后，用词更为正式，替换了极端情绪的表达，并表达了感谢。
- Arrr, I be fuming（呀，我气的发抖） 换成了 I am quite upset （我有点失望）
- And to make matters worse（更糟糕地是），换成了 Additionally(还有)
- I need yer help right now, matey!（我需要你的帮助），换成了Would you be able to assist me, please? Thank you kindly.（请问您能帮我吗？非常感谢您的好意）


✨ 你可以尝试修改提示，看可以得到什么不一样的结果😉

## Chat API：LangChain

在前面一部分，我们通过封装函数`get_completion`直接调用了OpenAI完成了对海岛邮件进行了翻译，得到用平和尊重的语气、美式英语表达的邮件。

让我们尝试使用LangChain来实现相同的功能。

In [36]:
# 如果你需要查看安装过程日志，可删除 -q 
# --upgrade 让我们可以安装到最新版本的 langchain
# !pip install -q --upgrade langchain

In [37]:
conda list langchain

# packages in environment at /home/wm/miniconda3/envs/py39:
#
# Name                    Version                   Build  Channel
langchain                 0.2.5                    pypi_0    pypi
langchain-community       0.0.27                   pypi_0    pypi
langchain-core            0.2.7                    pypi_0    pypi
langchain-text-splitters  0.2.1                    pypi_0    pypi

Note: you may need to restart the kernel to use updated packages.


In [38]:
# pip install langchain

### 模型<a id='model'></a>

从`langchain.chat_models`导入`OpenAI`的对话模型`ChatOpenAI`。 除去OpenAI以外，`langchain.chat_models`还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://python.langchain.com/en/latest/modules/models/chat/integrations.html)。

In [39]:
# from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [40]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
chat = OpenAI(temperature=0.0)
chat

OpenAI(client=<openai.resources.completions.Completions object at 0x7fb3dd746ca0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fb3dc048340>, temperature=0.0, openai_api_key='sk-wl6rN8KYKb7saRIbp3A6DCO93eMqj0bz2QibYGGn8QeGyzcc', openai_proxy='')

上面的输出显示ChatOpenAI的默认模型为`gpt-3.5-turbo`

### 提示模板

在前面的例子中，我们通过[f字符串](https://docs.python.org/zh-cn/3/tutorial/inputoutput.html#tut-f-strings)把Python表达式的值`style`和`customer_email`添加到`prompt`字符串内。

```python
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""
```
`langchain`提供了接口方便快速的构造和使用提示。现在我们来看看如何使用`langchain`来构造提示。

#### 📚 使用LongChain提示模版
##### 1️⃣ 构造提示模版字符串
我们构造一个提示模版字符串：`template_string`

In [41]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

##### 2️⃣ 构造LangChain提示模版
我们调用`ChatPromptTemplatee.from_template()`函数将上面的提示模版字符`template_string`转换为提示模版`prompt_template`

In [42]:
# 需要安装最新版的 LangChain
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [43]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

从上面的输出可以看出，`prompt_template` 有两个输入变量： `style` 和 `text`。

In [44]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

##### 3️⃣ 使用模版得到客户消息提示

langchain提示模版`prompt_template`需要两个输入变量： `style` 和 `text`。 这里分别对应 
- `customer_style`: 我们想要的顾客邮件风格
- `customer_email`: 顾客的原始邮件文本。

In [45]:
customer_style = """American English \
in a calm and respectful tone , at last translate to Simplified Chinese
"""

In [46]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

对于给定的`customer_style`和`customer_email`, 我们可以使用提示模版`prompt_template`的`format_messages`方法生成想要的客户消息`customer_messages`。

In [47]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [48]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


可以看出`customer_messages`变量类型为列表(`list`)，而列表里的元素变量类型为langchain自定义消息(`langchain.schema.HumanMessage`)。

打印第一个元素可以得到如下:

In [49]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone , at last translate to Simplified Chinese\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


##### 4️⃣ 调用chat模型转换客户消息风格

现在我们可以调用[模型](#model)部分定义的chat模型来实现转换客户消息风格。到目前为止，我们已经实现了在前一部分的任务。

In [68]:
# customer_response = chat(customer_messages)
customer_response = chat(prompt = customer_messages[0].content)

In [ ]:
print(customer_response.content)

I am quite upset that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add insult to injury, the warranty does not cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance at this time, my friend.

我很生气，因为我的搅拌机盖子飞了出去，把我的厨房墙壁弄得到处都是果汁！更糟糕的是，保修不包括清理厨房的费用。朋友，我现在需要你的帮助。


##### 5️⃣ 使用模版得到回复消息提示

接下来，我们更进一步，将客服人员回复的消息，转换为海盗的语言风格，并确保消息比较有礼貌。 

这里，我们可以继续使用第2️⃣步构造的langchain提示模版，来获得我们回复消息提示。

In [55]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [56]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\ , at last translate to Simplified Chinese
"""

In [65]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)
print(type(service_messages[0].content))

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate\ , at last translate to Simplified Chinese
. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```

<class 'str'>


##### 6️⃣ 调用chat模型转换回复消息风格

调用[模型](#model)部分定义的chat模型来转换回复消息风格

In [67]:
service_response = chat(service_messages[0].content)
print(service_response)


Ahoy there matey, the warranty be not coverin' the cost o' cleanin' yer galley fer ye, as it be yer own fault fer misusin' yer blender by forgettin' to put the lid on afore startin' it. Tough luck! Farewell!

嘿，顾客，保修不包括您厨房的清洁费用，因为您的错，您在启动搅拌机之前忘记盖上盖子。真倒霉！再见！


#### ❓为什么需要提示模版





在应用于比较复杂的场景时，提示可能会非常长并且包含涉及许多细节。使用提示模版，可以让我们更为方便地重复使用设计好的提示。

下面给出了一个比较长的提示模版案例。学生们线上学习并提交作业，通过以下的提示来实现对学生的提交的作业的评分。

```python
    prompt = """ Your task is to determine if the student's solution is correct or not

    To solve the problem do the following:
    - First, workout your own solution to the problem
    - Then compare your solution to the student's solution 
    and evaluate if the sudtent's solution is correct or not.
    ...
    Use the following format:
    Question:
    ```
    question here
    ```
    Student's solution:
    ```
    student's solution here
    ```
    Actual solution:
    ```
    ...
    steps to work out the solution and your solution here
    ```
    Is the student's solution the same as acutal solution \
    just calculated:
    ```
    yes or no
    ```
    Student grade
    ```
    correct or incorrect
    ```
    
    Question:
    ```
    {question}
    ```
    Student's solution:
    ```
    {student's solution}
    ```
    Actual solution:
    
    """
```

此外，LangChain还提供了提示模版用于一些常用场景。比如summarization, Question answering, or connect to sql databases, or connect to different APIs. 通过使用LongChain内置的提示模版，你可以快速建立自己的大模型应用，而不需要花时间去设计和构造提示。

最后，我们在建立大模型应用时，通常希望模型的输出为给定的格式，比如在输出使用特定的关键词来让输出结构化。 下面为一个使用大模型进行链式思考推理例子，对于问题：`What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?`, 通过使用LangChain库函数，输出采用"Thought"（思考）、"Action"（行动）、"Observation"（观察）作为链式思考推理的关键词，让输出结构化。在[补充材料](#reason_act)中，可以查看使用LangChain和OpenAI进行链式思考推理的另一个代码实例。

```python
"""
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions

Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United States)]
Observation: The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]

Thought: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.
Action: Finish[1,800 to 7,000 ft]

"""
```


### 输出解析器

#### 📚 如果没有输出解析器

对于给定的评价`customer_review`, 我们希望提取信息，并按以下格式输出：

```python
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}
```

In [72]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

##### 1️⃣ 构造提示模版字符串

In [73]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

##### 2️⃣ 构造langchain提示模版

In [74]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


##### 3️⃣ 使用模版得到提示消息

In [75]:
messages = prompt_template.format_messages(text=customer_review)

##### 4️⃣ 调用chat模型提取信息

In [77]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

/home/wm/miniconda3/envs/py39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/wm/miniconda3/envs/py39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


##### 📝 分析与总结
`response.content`类型为字符串（`str`），而并非字典(`dict`), 直接使用`get`方法会报错。因此，我们需要输出解释器。

In [78]:
type(response.content)

str

In [79]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

#### 📚 LangChain输出解析器

##### 1️⃣ 构造提示模版字符串

In [80]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

##### 2️⃣ 构造langchain提示模版

In [81]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

##### 🔥 构造输出解析器

In [86]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
#这个东西买了吗
# 作为给别人的礼物?\
# 如果是，回答True，\
# 如果不是或未知，则为False。

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days                                     did it take for the product\
                                      to arrive? If this information is not found,\
                                      output -1.")

# 这个产品用了多少天
# 到达呢?如果没有找到此信息，则\
# 输出-1。




price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")
# 价格

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                     did it take for the product                                      to arrive? If this information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


##### 3️⃣ 使用模版得到提示消息

In [87]:
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

In [88]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

##### 4️⃣ 调用chat模型提取信息

In [89]:
response = chat(messages)
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


##### 5️⃣ 使用输出解析器解析输出

In [90]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

##### 📝 分析与总结
`output_dict`类型为字典(`dict`), 可直接使用`get`方法。这样的输出更方便下游任务的处理。

In [91]:
type(output_dict)

dict

In [92]:
output_dict.get('delivery_days')

2

## 补充材料

### 链式思考推理(ReAct) <a id='reason_act'></a>
参考资料：[ReAct (Reason+Act) prompting in OpenAI GPT and LangChain](https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/)

In [93]:
!pip install -q wikipedia

In [1]:
conda list openai

# packages in environment at /home/wm/miniconda3/envs/langchain:
#
# Name                    Version                   Build  Channel
openai                    1.14.3                   pypi_0    pypi

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openai==0.28

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 699.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.14.3
    Uninstalling openai-1.14.3:
      Successfully uninstalled openai-1.14.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  )
]

# 使用大语言模型
llm = OpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0,
)

# 初始化ReAct代理
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
agent_executor = AgentExecutor.from_agent_and_tools(
  agent=react.agent,
  tools=tools,
  verbose=True,
)


question = "Author David Chanoff has collAuthor David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?aborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
agent_executor.run(question)

/home/wm/miniconda3/envs/langchain/lib/python3.9/site-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/wm/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0
  warn_deprecated(
/home/wm/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/home/wm/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_community/llms/openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer suppo



> Entering new AgentExecutor chain...


Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8dd623ad30>: Failed to establish a new connection: [Errno 101] Network is unreachable')).
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8dd6255c70>: Failed to establish a new connection: [Errno 101] Network is unreachable')).
Retrying langchain_community.llms.openai.completion_with_retry.<locals>._completion_with_retry

APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f8dd62638e0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))